# Generate new features to train model

## Import Packages

In [50]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from datetime import datetime
import time

## Get Directories

In [51]:
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
DATA_DIR = os.path.join(BASE_DIR, '../clean_data', 'data_out')

In [52]:
db_user = 'db_user'    # Replace with your PostgreSQL username
db_password = 'db_password'  # Replace with your PostgreSQL password
db_host = 'localhost'      # Replace with your PostgreSQL host (e.g., localhost or IP)
db_port = '5432'           # PostgreSQL port (default is 5432)
db_name = 'db_name'  # Replace with your PostgreSQL db name  

connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [53]:
# Create the SQLAlchemy engine
engine = create_engine(connection_string)
#Create query
query = "SELECT * FROM raw_match_data"
# Read the dfFrame from a PostgreSQL table
try:
    df = pd.read_sql(query, engine)

    print("Table queried successfully.")
    
finally:
    engine.dispose()
    print("Connection closed.")

Table queried successfully.
Connection closed.


In [54]:
df = df.iloc[::-1].reset_index(drop=True)


## Convert feature types

In [55]:
# Convert number columns to integers, data column to datetime
df['season'] = df['season'].str.split('/').str[0]
df['date'] = pd.to_datetime(df['date'])
df['match_name'] = df['home_team'] + ' - ' + df['away_team']
df[['season', 'home_goals', 'away_goals', 'home_odds', 'draw_odds', 'away_odds']] = df[['season', 'h_goals', 'a_goals', 'h_odds', 'd_odds', 'a_odds']].astype(np.int64)
df = df.drop(columns=['h_goals', 'a_goals', 'h_odds', 'd_odds', 'a_odds'])

## Add new features to matches

In [56]:
#Add winner column to denote result of match
df['winner'] = np.where(df['home_goals'] > df['away_goals'], 'HOME_TEAM', np.where(df['away_goals'] > df['home_goals'], 'AWAY_TEAM', 'DRAW'))

#Give home teams result points
df['home_match_points'] = np.where(df['winner'] == 'HOME_TEAM', 3 , np.where(df['winner'] == 'DRAW',1, 0))

#Give away teams result points
df['away_match_points'] = np.where(df['winner'] == 'AWAY_TEAM', 3 , np.where(df['winner'] == 'DRAW',1, 0))


## Calculate Last 3 averages

In [57]:
# Define a function to calculate simple and weighted exponential averages
def calculate_last_3_averages(matches):
    # Handle if fewer than 3 matches are available
    last_3_matches = matches[-3:]  # Get the last 3 matches
    
    # If there are no matches yet, return zeros
    if not last_3_matches:
        return 0, 0, 0, 0, 0, 0
    
    # Calculate simple averages
    avg_pts = round(sum(m['points'] for m in last_3_matches) / len(last_3_matches), 3)
    avg_goals_for = round(sum(m['goals_for'] for m in last_3_matches) / len(last_3_matches), 3)
    avg_goals_against = round(sum(m['goals_against'] for m in last_3_matches) / len(last_3_matches), 3)
    
    # Calculate weighted averages with higher weight for more recent matches
    weights = [0.5, 0.3, 0.2]  # Example weights for the last 3 matches (most recent gets highest weight)
    weight_sum = sum(weights[:len(last_3_matches)])
    
    wavg_pts = round(sum(m['points'] * weights[i] for i, m in enumerate(last_3_matches)) / weight_sum, 3)
    wavg_goals_for = round(sum(m['goals_for'] * weights[i] for i, m in enumerate(last_3_matches)) / weight_sum, 3)
    wavg_goals_against = round(sum(m['goals_against'] * weights[i] for i, m in enumerate(last_3_matches)) / weight_sum, 3)
    
    return avg_pts, avg_goals_for, avg_goals_against, wavg_pts, wavg_goals_for, wavg_goals_against

## Update teams season stats for current match

In [58]:
# Define a function to update team stats, and append match data to the last 3 matches list
def update_team_stats(stats, team, is_home, goals_for, goals_against, result, opponent_rank):
    # Update points and streaks based on match result
    if result == 'win':
        points = 3
        stats[team]['points'] += 3
        stats[team]['wins'] += 1
        stats[team]['win_streak'] += 1
        stats[team]['loss_streak'] = 0
        stats[team]['draw_streak'] = 0
    elif result == 'draw':
        points = 1
        stats[team]['points'] += 1
        stats[team]['draws'] += 1
        stats[team]['draw_streak'] += 1
        stats[team]['win_streak'] = 0
        stats[team]['loss_streak'] = 0
    else:  # loss
        points = 0
        stats[team]['losses'] += 1
        stats[team]['loss_streak'] += 1
        stats[team]['win_streak'] = 0
        stats[team]['draw_streak'] = 0
    
    # Append current match data to the last_3_matches list
    stats[team]['last_3_matches'].append({
        'points': points,
        'goals_for': goals_for,
        'goals_against': goals_against,
        'opponent_rank': opponent_rank
    })

    stats[team]['all_matches'].append({
        'points': points,
        'goals_for': goals_for,
        'goals_against': goals_against,
        'opponent_rank': opponent_rank
    })
    
    # Limit the list to the last 3 matches
    if len(stats[team]['last_3_matches']) >3 :
        stats[team]['last_3_matches'].pop(0)
    

    
    # Update cumulative stats
    stats[team]['goals_for'] += goals_for
    stats[team]['goals_against'] += goals_against

In [59]:
def add_team_stats(stats, team):
    if team not in stats:
        stats[team] = {
            'points': 0,
            'goals_for': 0,
            'goals_against': 0,
            'wins': 0,
            'draws': 0,
            'losses': 0,
            'win_streak': 0,
            'loss_streak': 0,
            'draw_streak': 0,
            'last_3_matches': [], # Store last 3 matches for calculating averages
            'all_matches': []
        }


## Iterate over matches df, adding features for each entry

In [63]:
team_stats = {}
ranks = {}
# Define the rank difference threshold to determine similar ranks
similar_rank_threshold = 2

# Iterate over each row in the DataFrame
for idx, row in df.iterrows():
    season = row['season']
    home_team = row['home_team']
    away_team = row['away_team']
    home_goals = row['home_goals']
    away_goals = row['away_goals']
    
    if season not in team_stats:
        team_stats[season] = {}
    
    # Determine match result for both home and away teams
    if home_goals > away_goals:
        home_result = 'win'
        away_result = 'loss'
    elif home_goals < away_goals:
        home_result = 'loss'
        away_result = 'win'
    else:
        home_result = 'draw'
        away_result = 'draw'
    
    # Ensure each team's stats are initialized
    add_team_stats(team_stats[season], home_team)
    add_team_stats(team_stats[season], away_team)
    
    # Calculate last 3 averages for home and away teams
    home_avg_pts, home_avg_goals, home_avg_goals_against, home_wavg_pts, home_wavg_goals, home_wavg_goals_against = calculate_last_3_averages(team_stats[season][home_team]['last_3_matches'])
    away_avg_pts, away_avg_goals, away_avg_goals_against, away_wavg_pts, away_wavg_goals, away_wavg_goals_against = calculate_last_3_averages(team_stats[season][away_team]['last_3_matches'])
    
    # Calculate rank by sorting the teams based on points, then goal difference
    standings = sorted(team_stats[season].items(), key=lambda x: (x[1]['points'], x[1]['goals_for'] - x[1]['goals_against'], x[1]['goals_for']), reverse=True)
    ranks = {team: rank+1 for rank, (team, stats) in enumerate(standings)}
    
    # Attributes to track for each match
    attributes = [
        'points', 'rank', 'goals_for', 'goals_against', 
        'wins', 'draws', 'losses', 'win_streak', 
        'loss_streak', 'draw_streak'
    ]
    
    # Update DataFrame for home and away teams
    for attr in attributes:
        if attr == 'rank':
            df.at[idx, 'home_rank'] = ranks[home_team]
            df.at[idx, 'away_rank'] = ranks[away_team]
        else:
            df.at[idx, f'home_{attr}'] = team_stats[season][home_team][attr]
            df.at[idx, f'away_{attr}'] = team_stats[season][away_team][attr]

    # Add calculated averages to the DataFrame
    averages = [
        ('avg_pts', home_avg_pts, away_avg_pts),
        ('goals', home_avg_goals, away_avg_goals),
        ('goals_against', home_avg_goals_against, away_avg_goals_against),
        ('wavg_pts', home_wavg_pts, away_wavg_pts),
        ('wavg_goals', home_wavg_goals, away_wavg_goals),
        ('wavg_goals_against', home_wavg_goals_against, away_wavg_goals_against)
    ]

    for suffix, home_val, away_val in averages:
        df.at[idx, f'home_last_3_{suffix}'] = home_val
        df.at[idx, f'away_last_3_{suffix}'] = away_val

    # Calculate and add the home_points * away_points interaction
    if team_stats[season][away_team]['points'] == 0 and team_stats[season][home_team]['points'] == 0:
        df.at[idx, 'home_away_points_interaction'] = 1
    elif team_stats[season][away_team]['points'] == 0:
        df.at[idx, 'home_away_points_interaction'] = team_stats[season][home_team]['points']
    else: 
        df.at[idx, 'home_away_points_interaction'] = team_stats[season][home_team]['points'] / team_stats[season][away_team]['points']
    
    # Calculate win/loss ratio against similarly ranked teams for home and away teams
    home_similar_rank_matches = 0
    home_similar_rank_wins = 0
    home_similar_rank_goals = 0
    home_similar_rank_ga = 0
    away_similar_rank_matches = 0
    away_similar_rank_wins = 0
    away_similar_rank_goals = 0
    away_similar_rank_ga = 0

    # Check previous match results to calculate ratios against similarly ranked teams
    for match in team_stats[season][home_team]['all_matches']:
        opponent_rank = ranks[away_team]
        if abs(match['opponent_rank'] - opponent_rank) <= similar_rank_threshold:
            home_similar_rank_matches += 1
            if match['points'] == 3:
                home_similar_rank_wins += 1
                home_similar_rank_goals += match['goals_for']
                home_similar_rank_ga += match['goals_against']

    for match in team_stats[season][away_team]['all_matches']:
        opponent_rank = ranks[home_team]
        if abs(match['opponent_rank'] - opponent_rank) <= similar_rank_threshold:
            away_similar_rank_matches += 1
            if match['points'] == 3:
                away_similar_rank_wins += 1
                away_similar_rank_goals += match['goals_for']
                away_similar_rank_ga += match['goals_against']

    # Calculate and store win/loss ratios in the DataFrame
    df.at[idx, 'home_similar_rank_win_ratio'] = (home_similar_rank_wins / home_similar_rank_matches) if home_similar_rank_matches > 0 else 0
    df.at[idx, 'away_similar_rank_win_ratio'] = (away_similar_rank_wins / away_similar_rank_matches) if away_similar_rank_matches > 0 else 0
    df.at[idx, 'home_similar_rank_goals'] = (home_similar_rank_goals / home_similar_rank_matches) if home_similar_rank_matches > 0 else 0
    df.at[idx, 'away_similar_rank_goals'] = (away_similar_rank_goals / away_similar_rank_matches) if away_similar_rank_matches > 0 else 0
    df.at[idx, 'home_similar_rank_ga'] = (home_similar_rank_ga / home_similar_rank_matches) if home_similar_rank_matches > 0 else 0
    df.at[idx, 'away_similar_rank_ga'] = (away_similar_rank_ga / away_similar_rank_matches) if away_similar_rank_matches > 0 else 0
    df.at[idx, 'home_similar_rank_goal_ratio'] = (home_similar_rank_goals / home_similar_rank_ga) if home_similar_rank_ga > 0 else 1
    df.at[idx, 'away_similar_rank_goal_ratio'] = (away_similar_rank_goals / away_similar_rank_ga) if away_similar_rank_ga > 0 else 1

    # Update home and away team stats
    update_team_stats(team_stats[season], home_team, True, home_goals, away_goals, home_result, ranks[away_team])
    update_team_stats(team_stats[season], away_team, False, away_goals, home_goals, away_result, ranks[home_team])

    standings = sorted(team_stats[season].items(), key=lambda x: (x[1]['points'], x[1]['goals_for'] - x[1]['goals_against'], x[1]['goals_for']), reverse=True)
    ranks = {team: rank+1 for rank, (team, stats) in enumerate(standings)}

## Send data to Postgres DB

In [66]:
ranks_df = pd.DataFrame(list(ranks.items()), columns=['team_name', 'rank'])


# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to a PostgreSQL table
try:
    # Write the DataFrame to a table named 'your_table_name', replace it with your desired table name
    df.to_sql('match_statistics', engine, index=False, if_exists='replace')
    ranks_df.to_sql('team_rankings', engine, index=False, if_exists='replace')

    print("Table created successfully.")
except Exception as e:
    print(f"Error: {e}")

Table created successfully.


## Prep team info and match info for DB Entry

In [67]:
# Flatten the data structure
teams = []
matches = []
current_year = datetime.now().year if datetime.now().month > 6 else (datetime.now().year - 1)

for team_name, team_data in team_stats[current_year].items():
    # Team-level data
    team_entry = {
        'team_name': team_name,
        'points': team_data['points'],
        'goals_for': team_data['goals_for'],
        'goals_against': team_data['goals_against'],
        'wins': team_data['wins'],
        'draws': team_data['draws'],
        'losses': team_data['losses'],
        'win_streak': team_data['win_streak'],
        'loss_streak': team_data['loss_streak'],
        'draw_streak': team_data['draw_streak']
    }
    teams.append(team_entry)

    # Match-level data
    for match_type, match_list in [('last_3_matches', team_data['last_3_matches']), ('all_matches', team_data['all_matches'])]:
        for match in match_list:
            match_entry = {
                'team_name': team_name,
                'match_type': match_type,
                'points': match['points'],
                'goals_for': match['goals_for'],
                'goals_against': match['goals_against'],
                'opponent_rank': match['opponent_rank']
            }
            matches.append(match_entry)


## Send match and team info to Postgres

In [68]:
conn_details = {
    'dbname': 'drewf',
    'user': 'drewf',
    'password': 'Soccer.666',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the database
conn = psycopg2.connect(**conn_details)
cur = conn.cursor()

# Insert teams data
cur.execute("DROP TABLE IF EXISTS teams CASCADE;")
cur.execute("""
        CREATE TABLE teams (
            team_name VARCHAR PRIMARY KEY,
            points INT,
            goals_for INT,
            goals_against INT,
            wins INT,
            draws INT,
            losses INT,
            win_streak INT,
            loss_streak INT,
            draw_streak INT
        );
    """)
for team in teams:
    cur.execute(
        """
        INSERT INTO teams (team_name, points, goals_for, goals_against, wins, draws, losses, win_streak, loss_streak, draw_streak)
        VALUES (%(team_name)s, %(points)s, %(goals_for)s, %(goals_against)s, %(wins)s, %(draws)s, %(losses)s, %(win_streak)s, %(loss_streak)s, %(draw_streak)s)
        ON CONFLICT (team_name) DO NOTHING;
        """,
        team
    )

# Insert matches data
cur.execute("DROP TABLE IF EXISTS matches;")
cur.execute("""
        CREATE TABLE matches (
            match_id SERIAL PRIMARY KEY,
            team_name VARCHAR REFERENCES teams (team_name),
            match_type VARCHAR,
            points INT,
            goals_for INT,
            goals_against INT,
            opponent_rank INT
        );
    """)
for match in matches:
    cur.execute(
        """
        INSERT INTO matches (team_name, match_type, points, goals_for, goals_against, opponent_rank)
        VALUES (%(team_name)s, %(match_type)s, %(points)s, %(goals_for)s, %(goals_against)s, %(opponent_rank)s);
        """,
        match
    )

# Commit and close connection
conn.commit()
cur.close()
conn.close()